In [65]:
using OhMyJulia
using DataFrames

## Setup

In [86]:
function parse_path(file)
    pathdict = Dict{Pair{String, String}, Vector{Pair{Vector{Pair{String, String}}, Float64}}}()
    
    cp = nothing
    for line in eachline(file)
        m = match(r"([hs]\d+)\s*->\s*([hs]\d+)", line)
        if m != nothing
            pathdict[car(m) => cadr(m)] = []
            cp = pathdict[car(m) => cadr(m)]
            continue
        end
        
        s = split(line, '@')
        if length(s) == 2
            p = parse(car(s))
            p = map(x->Pair(string.(x.args)...), p.args)
            w = parse(cadr(s))
            push!(cp, p => w)
        end
    end
    
    pathdict
end

function parse_topo(file)
    nodes = Set{String}()
    edges = Set{Pair{String, String}}()
    for line in eachline(file)
        m = match(r"([hs]\d+)\s*->\s*([hs]\d+)", line)
        m == nothing && continue
        
        src, dst = car(m), cadr(m)
        push!(nodes, src, dst)
        push!(edges, src => dst)
    end
    collect(nodes), collect(edges)
end

function mkdf(names...)
    DataFrame([[] for i in names], [Symbol(i) for i in names])
end

function cut_path()
    ref = Dict(k=>Dict(length(car(ps)) for (pair, ps) in v) for (k, v) in paths["custom_0"])
    
end

data = "abilene"
nodes, edges = parse_topo("data/topologies/$data.dot")
# algos = [basename(x) for x in all_files("data/results/$data/paths") if match(r"_\d+$", x) != nothing]
algos = [basename(x) for x in all_files("data/results/$data/paths") if endswith(x, "_0")]
paths = Dict(algo => parse_path("data/results/$data/paths/$algo") for algo in algos)
#cut_path()

Dict{String,Dict{Pair{String,String},Array{Pair{Array{Pair{String,String},1},Float64},1}}} with 2 entries:
  "semimcfraeke_0"  => Dict(("h10"=>"h9")=>Pair{Array{Pair{String,String},1},Fl…
  "semimcfcustom_0" => Dict(("h10"=>"h9")=>Pair{Array{Pair{String,String},1},Fl…

## avg path length

In [76]:
df = mkdf("algorithm", "average length", "max length")
for (algo, path) in paths
    lens = [length(car(p)) for (k, v) in path for p in v]
    push!(df, [algo[1:end-2] mean(lens) maximum(lens)])
end
df

,algorithm,average length,max length
1,custom,5.25,9
2,semimcfvlb,6.31707,11
3,raeke,7.43234,13
4,semimcfecmp,4.7619,7
5,semimcfraeke,7.2912,12
6,akraeke,7.37906,12
7,akvlb,6.33936,11
8,akecmp,4.7619,7
9,ksp,7.99423,13
10,vlb,6.34686,12


## one edge failure

In [83]:
df = mkdf("algorithm", "affected path", "affected pair", "affected weight")
for (algo, path) in paths
    apath, apair, aweight = 0, 0, 0
    
    for edge in edges @when startswith(car(edge), "s") && startswith(cadr(edge), "s")
        for (pair, scheme) in path
            pair_affected = false
            
            for (p, w) in scheme @when edge in p
                pair_affected = true
                apath += 1
                aweight += w
            end
            
            apair += pair_affected
        end
    end
    
    push!(df, [algo[1:end-2] apath apair aweight])
end
df

,algorithm,affected path,affected pair,affected weight
1,custom,806,806,427.333
2,semimcfvlb,2124,1444,482.204
3,raeke,4536,1823,450.202
4,semimcfecmp,464,416,330.0
5,semimcfraeke,4270,1735,510.711
6,akraeke,4470,1859,445.992
7,akvlb,2161,1449,431.0
8,akecmp,464,416,330.0
9,ksp,6234,1944,751.959
10,vlb,2143,1444,430.333


## one node failure

In [84]:
df = mkdf("algorithm", "affected path", "affected pair", "affected weight")
for (algo, path) in paths
    apath, apair, aweight = 0, 0, 0
    
    for node in nodes @when startswith(node, "s")
        for (pair, scheme) in path
            pair_affected = false
            
            for (p, w) in scheme @when any(x->node in x, p)
                pair_affected = true
                apath += 1
                aweight += w
            end
            
            apair += pair_affected
        end
    end
    
    push!(df, [algo[1:end-2] apath apair aweight])
end
df

,algorithm,affected path,affected pair,affected weight
1,custom,1054,822,559.333
2,semimcfvlb,2616,1202,614.204
3,raeke,5371,1382,582.202
4,semimcfecmp,632,512,462.0
5,semimcfraeke,5077,1328,642.711
6,akraeke,5301,1410,577.992
7,akvlb,2659,1202,563.0
8,akecmp,632,512,462.0
9,ksp,7274,1454,883.959
10,vlb,2636,1206,562.333
